In [16]:
import torch
import json
import pandas as pd
from diffusers import LTXPipeline

In [41]:
model_id = "a-r-r-o-w/LTX-Video-0.9.1-diffusers"
LORA_WEIGHT = 1.0
device="cuda:1"

pipe = LTXPipeline.from_pretrained(model_id, torch_dtype=torch.bfloat16, local_files_only=True)

lora_path = "/mnt/ssd0/saksham/i2av/ltx_lora_training_i2v_t2v/audioldm_lora/checkpoint-10000"
pipe.load_lora_weights(lora_path, weight_name="pytorch_lora_weights.safetensors", adapter_name="ltx_lora")
pipe.set_adapters("ltx_lora", LORA_WEIGHT)
# ----------
_ = pipe.to(device)

Loading pipeline components...: 100%|██████████| 5/5 [00:01<00:00,  2.78it/s]


In [42]:
meta_path = '/mnt/ssd0/saksham/i2av/AVSync15/metadata.csv'
df = pd.read_csv(meta_path)
path = '/mnt/ssd0/saksham/i2av/AVSync15/aud_caption.json'
data = json.load(open(path))

In [55]:
negative_prompt = ""
prefix = "sounding object, "

label = 'baby_babbling_crying'
caption = ' A baby is crying and making sounds.'

prompt = prefix + f'{label}, {caption}'

video = pipe(
    prompt=prompt,
    negative_prompt=negative_prompt,
    width=128*32,
    height=16*32,
    num_frames=1,
    num_inference_steps=50,
    decode_timestep=0.03,
    decode_noise_scale=0.025,
    output_type='latent'
).frames


100%|██████████| 50/50 [00:11<00:00,  4.27it/s]


In [44]:
def _unpack_latents(
        latents: torch.Tensor, num_frames: int, height: int, width: int, patch_size: int = 1, patch_size_t: int = 1
    ) -> torch.Tensor:
    batch_size = latents.size(0)
    latents = latents.reshape(batch_size, num_frames, height, width, -1, patch_size_t, patch_size, patch_size)
    latents = latents.permute(0, 4, 1, 5, 2, 6, 3, 7).flatten(6, 7).flatten(4, 5).flatten(2, 3)
    return latents

def _normalize_latents(
    latents: torch.Tensor, latents_mean: torch.Tensor, latents_std: torch.Tensor, scaling_factor: float = 1.0,
    reverse=False,
) -> torch.Tensor:
    # Normalize latents across the channel dimension [B, C, F, H, W]
    latents_mean = latents_mean.view(1, -1, 1, 1, 1).to(latents.device, latents.dtype)
    latents_std = latents_std.view(1, -1, 1, 1, 1).to(latents.device, latents.dtype)
    if not reverse:
        latents = (latents - latents_mean) * scaling_factor / latents_std
    else:
        latents = latents * latents_std / scaling_factor + latents_mean
    return latents

In [45]:
from ltx_video_lora import load_latent_models

In [22]:
device = "cuda:1"
dtype = torch.bfloat16
vae = load_latent_models()["vae"].to(device, dtype=dtype)

In [46]:
ll = video

In [52]:
num_frames = 1; height = 16; width = 128
device = "cuda"; dtype = torch.bfloat16
lt = _unpack_latents(ll.to(device, dtype=dtype), 1, 16, 128)


In [48]:
lt.shape

torch.Size([1, 128, 1, 16, 128])

In [53]:
lt1 = lt[0].permute(1,0,3,2)
lt1 = lt1[:,:8]
lt1 = _normalize_latents(lt1, vae.latents_mean, vae.latents_std, reverse=True)
lt1.shape

torch.Size([1, 128, 8, 128, 16])

In [27]:
import sys

path = 'audioldm_vae/'
sys.path.append(path)
from aud_utils import AudioLDM_VAE
from IPython.display import Audio

In [37]:
vae_obj = AudioLDM_VAE()

Fetching 9 files: 100%|██████████| 9/9 [00:00<00:00, 127529.51it/s]


In [54]:
rec_audio = vae_obj.latent_to_audio(lt1.float())

RuntimeError: Expected 3D (unbatched) or 4D (batched) input to conv2d, but got input of size: [1, 128, 8, 128, 16]

In [51]:
Audio(rec_audio, rate=16000)